# Compartilhamento de Clipboard em Rede
Este notebook implementa um sistema que permite compartilhar o conteúdo do clipboard entre computadores na mesma rede.
Necessário instalar as seguintes bibliotecas:

In [ ]:

```

In [ ]:
from flask import Flask, jsonify, request
import pyperclip
import threading

app = Flask(__name__)
last_clipboard = ""

@app.route('/get_clipboard', methods=['GET'])
def get_clipboard():
    return jsonify({'content': last_clipboard})

@app.route('/set_clipboard', methods=['POST'])
def set_clipboard():
    global last_clipboard
    data = request.json
    last_clipboard = data.get('content', '')
    pyperclip.copy(last_clipboard)
    return jsonify({'status': 'success'})

def run_server():
    app.run(host='0.0.0.0', port=5000

In [ ]:
import time
import requests
import pyperclip

def monitor_clipboard():
    last_content = pyperclip.paste()

    while True:
        current_content = pyperclip.paste()
        if current_content != last_content:
            try:
                requests.post('http://localhost:5000/set_clipboard',
                            json={'content': current_content})
                last_content = current_content
            except:
                pass
        time.sleep(1

In [ ]:
if __name__ == '__main__':
    # Inicia o servidor em uma thread separada
    server_thread = threading.Thread(target=run_server)
    server_thread.daemon = True
    server_thread.start()

    # Inicia o monitor do clipboard
    monitor_clipboard()

# Como usar
1. Execute este script no computador que será o servidor
2. Para outros computadores se conectarem, modifique o endereço no código do monitor para o IP do servidor
3. Quando algo for copiado (Ctrl+C) em qualquer computador conectado, será sincronizado com os demais

In [ ]:
import pyperclip
import requests
import time

def monitor_remote_clipboard(server_ip):
    last_content = ""
    while True:
        try:
            response = requests.get(f'http://{server_ip}:5000/get_clipboard')
            current_content = response.json()['content']

            if current_content != last_content:
                pyperclip.copy(current_content)
                last_content = current_content

        except:
            pass
        time.sleep(1)

if __name__ == '__main__':
    SERVER_IP = '192.168.1.100'  # Substitua pelo IP do servidor
    monitor_remote_clipboard(SERVER_IP)